In [5]:
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import numpy as np
import math

## Dataset preparation
dataset = [ 
    "မြန်မာ ပြည် ရှေ့ရေး အကြောင်း ဆွေးနွေးသွားတာ အရမ်း ကောင်းတယ်။", 
    "အရမ်း မှန်တဲ့ စကားတွေပါ။ ကျေးဇူး တင်ပါတယ်။", 
    "တန်ဖိုး ရှိတဲ့ စကားတွေပါ။ လေးစားပါတယ်။", 
    "သုံးသပ်သွားတာ ကောင်းလိုက်တာ။ အရမ်း ဗဟုသုတ ရတယ်။", 
    "ကျေးဇူးပါ ဆရာ သုံးသပ်ချက် တွေကို ကျေနပ် အားရပါတယ်။", 

    "မင်းဆိုး မင်းညစ် အယုတ်တမာ ကျဆုံးမှာပါ။", 
    "ကလိမ်ကကျစ် နဲ့ စကား ကို ဝေ့ဝိုက်ပြီး ပြောနေတယ်။", 
    "မိုက်ရိုက်လိုက်တဲ့ လုပ်ရပ်ဗျာ။ အတော် ဆိုးတာပဲ။", 
    "ကျက်သရေ တုံးပါကွာ။ အရည်အချင်းကို မရှိဘူး။", 
    "မင်းကို မြင်ရတာ စိတ် ကသိကအောက် ဖြစ်တယ်။", 

    "ပြည်ထောင်စု သမ္မတ မြန်မာ နိုင်ငံတော် ထာဝရ တည်တံ့နိုင်ပါစေ။", 
    "ပြည်ထောင်စု ဖွား တိုင်းရင်းသားတွေ စုစည်းနေရင် အရမ်း ကောင်းမှာပဲ။", 
    "စစ်ပွဲတွေ ကို မုန်းလိုက်တာ ဗျာ။", 
    "စာသင်ကျောင်း တံခါးများ အမြဲတမ်း ဖွင့်ထားနိုင်ပါစေ။", 
    "ရခိုင် မုန့်တီ၊ ရှမ်း ခေါက်ဆွဲ နဲ့ ကရင် ကုတိယို ကို အရမ်း ကြိုက်တယ်။" 
] 
targets = ["Positive", "Positive", "Positive", "Positive", "Positive", "Negative", "Negative", "Negative", "Negative", "Negative", "Neutral", "Neutral", "Neutral", "Neutral", "Neutral"]

## Tokenization
tokenized_dataset = [doc.replace('၊', '').replace('။', '').lower().split() for doc in dataset]
print(f'Dataset Length : {len(tokenized_dataset)}')
print(f'Tokenized Text : {tokenized_dataset[:5]}')

Dataset Length : 15
Tokenized Text : [['မြန်မာ', 'ပြည်', 'ရှေ့ရေး', 'အကြောင်း', 'ဆွေးနွေးသွားတာ', 'အရမ်း', 'ကောင်းတယ်'], ['အရမ်း', 'မှန်တဲ့', 'စကားတွေပါ', 'ကျေးဇူး', 'တင်ပါတယ်'], ['တန်ဖိုး', 'ရှိတဲ့', 'စကားတွေပါ', 'လေးစားပါတယ်'], ['သုံးသပ်သွားတာ', 'ကောင်းလိုက်တာ', 'အရမ်း', 'ဗဟုသုတ', 'ရတယ်'], ['ကျေးဇူးပါ', 'ဆရာ', 'သုံးသပ်ချက်', 'တွေကို', 'ကျေနပ်', 'အားရပါတယ်']]


In [6]:
## Label Encoder
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(targets)
print("Encoded Labels:", encoded_labels)
print("Classes:", label_encoder.classes_)

Encoded Labels: [2 2 2 2 2 0 0 0 0 0 1 1 1 1 1]
Classes: ['Negative' 'Neutral' 'Positive']


In [7]:
## BM25 Calculation Parameters
k1 = 1.5  # term frequency saturation parameter
b = 0.75  # length normalization parameter

## BM25 Functions
def compute_tf(doc):
    term_count = Counter(doc)
    return term_count

def compute_df(corpus):
    df = Counter()
    for doc in corpus:
        unique_terms = set(doc)
        for term in unique_terms:
            df[term] += 1
    return df

def compute_bm25(tf, df, corpus_size, avg_doc_len, doc_len, k1=1.5, b=0.75):
    bm25 = {}
    for term, term_freq in tf.items():
        df_term = df.get(term, 0)
        idf = math.log((corpus_size - df_term + 0.5) / (df_term + 0.5) + 1)  # BM25 IDF
        norm_tf = (term_freq * (k1 + 1)) / (term_freq + k1 * (1 - b + b * (doc_len / avg_doc_len)))
        bm25[term] = norm_tf * idf
    return bm25

In [8]:
## Calculate the document frequencies for the entire corpus
df = compute_df(tokenized_dataset)
print(f'DF for the entire corpus : {df}')

DF for the entire corpus : Counter({'အရမ်း': 5, 'ကို': 3, 'မြန်မာ': 2, 'စကားတွေပါ': 2, 'နဲ့': 2, 'ပြည်ထောင်စု': 2, 'ဆွေးနွေးသွားတာ': 1, 'ပြည်': 1, 'ရှေ့ရေး': 1, 'အကြောင်း': 1, 'ကောင်းတယ်': 1, 'တင်ပါတယ်': 1, 'မှန်တဲ့': 1, 'ကျေးဇူး': 1, 'လေးစားပါတယ်': 1, 'ရှိတဲ့': 1, 'တန်ဖိုး': 1, 'ဗဟုသုတ': 1, 'ကောင်းလိုက်တာ': 1, 'ရတယ်': 1, 'သုံးသပ်သွားတာ': 1, 'ဆရာ': 1, 'ကျေးဇူးပါ': 1, 'အားရပါတယ်': 1, 'တွေကို': 1, 'သုံးသပ်ချက်': 1, 'ကျေနပ်': 1, 'မင်းညစ်': 1, 'အယုတ်တမာ': 1, 'မင်းဆိုး': 1, 'ကျဆုံးမှာပါ': 1, 'စကား': 1, 'ဝေ့ဝိုက်ပြီး': 1, 'ကလိမ်ကကျစ်': 1, 'ပြောနေတယ်': 1, 'လုပ်ရပ်ဗျာ': 1, 'မိုက်ရိုက်လိုက်တဲ့': 1, 'ဆိုးတာပဲ': 1, 'အတော်': 1, 'တုံးပါကွာ': 1, 'ကျက်သရေ': 1, 'အရည်အချင်းကို': 1, 'မရှိဘူး': 1, 'ကသိကအောက်': 1, 'စိတ်': 1, 'မြင်ရတာ': 1, 'ဖြစ်တယ်': 1, 'မင်းကို': 1, 'နိုင်ငံတော်': 1, 'တည်တံ့နိုင်ပါစေ': 1, 'သမ္မတ': 1, 'ထာဝရ': 1, 'တိုင်းရင်းသားတွေ': 1, 'ဖွား': 1, 'စုစည်းနေရင်': 1, 'ကောင်းမှာပဲ': 1, 'စစ်ပွဲတွေ': 1, 'မုန်းလိုက်တာ': 1, 'ဗျာ': 1, 'စာသင်ကျောင်း': 1, 'အမြဲတမ်း': 1, 'ဖွင့်ထားနိုင်ပါစေ': 1, 'တံခါးမ

In [9]:
## Compute average document length
avg_doc_len = sum(len(doc) for doc in tokenized_dataset) / len(tokenized_dataset)

## Compute BM25 for each document
bm25_dataset = []
for doc in tokenized_dataset:
    tf = compute_tf(doc)
    bm25 = compute_bm25(tf, df, len(tokenized_dataset), avg_doc_len, len(doc), k1, b)
    bm25_dataset.append(bm25)
    
print(f'BM25 feature of index 0 : {bm25_dataset[0]}')

BM25 feature of index 0 : {'မြန်မာ': 1.627439334019179, 'ပြည်': 2.0752864562249793, 'ရှေ့ရေး': 2.0752864562249793, 'အကြောင်း': 2.0752864562249793, 'ဆွေးနွေးသွားတာ': 2.0752864562249793, 'အရမ်း': 0.9361890454806409, 'ကောင်းတယ်': 2.0752864562249793}


In [10]:
## Convert BM25 dictionary to feature vectors
vocabulary = list(df.keys())
def vectorize(bm25, vocab):
    return [bm25.get(term, 0.0) for term in vocab]

features = [vectorize(bm25, vocabulary) for bm25 in bm25_dataset]
print(f'Vocab Size : {vocabulary.__len__()}')
print(f'10th Sample : {dataset[10]}')
print(f'BM25 Features 10th sample : {features[10]}')

Vocab Size : 70
10th Sample : ပြည်ထောင်စု သမ္မတ မြန်မာ နိုင်ငံတော် ထာဝရ တည်တံ့နိုင်ပါစေ။
BM25 Features 10th sample : [0.0, 0.0, 0.0, 0.0, 1.7574418843698236, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.7574418843698236, 2.241063776692655, 2.241063776692655, 2.241063776692655, 2.241063776692655, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [12]:
## Classifier
def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    return dot_product / (norm_A * norm_B)

similarity = cosine_similarity(features[10], features[11])
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.12250406971052717


In [20]:
## Inference
test = "မင်းဆိုး မင်းညစ် အယုတ်တမာ တို့သည် ကျရှုံးကြစမြဲ ဖြစ်သည်။"
test_tf = compute_tf(test.replace('၊', '').replace('။', '').lower().split())
test_bm25 = compute_bm25(test_tf, df, len(tokenized_dataset), avg_doc_len, len(test_tf), k1, b)
test_vector = vectorize(test_bm25, vocabulary)
preds = [cosine_similarity(source, test_vector) for source in features]
idx = encoded_labels[np.argmax(preds)]
print(f'Predicted Label : {label_encoder.classes_[idx]}')

Predicted Label : Negative
